In [1]:
import os
import tensorflow as tf

print(tf.version.VERSION)

2.1.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
## Save checkpoints during training

In [4]:


checkpoint_path = "trainin_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)
model.fit(
    train_images, 
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 704/1000 [====================>.........] - ETA: 0s - loss: 1.3171 - accuracy: 0.6236
Epoch 00001: saving model to trainin_1/cp.ckpt
1000/1000 [==============================] - 0s 470us/sample - loss: 1.1290 - accuracy: 0.6690 - val_loss: 0.7626 - val_accuracy: 0.7630
Epoch 2/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.4399 - accuracy: 0.8835
Epoch 00002: saving model to trainin_1/cp.ckpt
1000/1000 [==============================] - 0s 157us/sample - loss: 0.4170 - accuracy: 0.8920 - val_loss: 0.5380 - val_accuracy: 0.8420
Epoch 3/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.2899 - accuracy: 0.9162
Epoch 00003: saving model to trainin_1/cp.ckpt
1000/1000 [==============================] - 0s 153us/sample - loss: 0.2846 - accuracy: 0.9210 - val_loss: 0.4623 - val_accuracy: 0.8630
Epoch 4/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0.1806 - accuracy: 0.9613
Epoch 00

In [5]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [7]:
untrained_model = create_model()
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print(f"Untrained model, accuracy: {100*acc}")

1000/1000 - 0s - loss: 2.4251 - accuracy: 0.0640
Untrained model, accuracy: 6.400000303983688


In [8]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Trained model, accuracy: {100*acc}")

1000/1000 - 0s - loss: 0.4153 - accuracy: 0.8680
Trained model, accuracy: 86.79999709129333


In [9]:
untrained_model.load_weights(checkpoint_path)

loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print(f"Trained model, accuracy: {100*acc}")

1000/1000 - 0s - loss: 0.4153 - accuracy: 0.8680
Trained model, accuracy: 86.79999709129333


## Checkpoint Options

In [11]:
checkpoint_path = "trainin_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)
model = create_model()
model.fit(
    train_images, 
    train_labels,
    epochs=50,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback],
    verbose=0
)


Epoch 00005: saving model to trainin_2/cp-0005.ckpt

Epoch 00010: saving model to trainin_2/cp-0010.ckpt

Epoch 00015: saving model to trainin_2/cp-0015.ckpt

Epoch 00020: saving model to trainin_2/cp-0020.ckpt

Epoch 00025: saving model to trainin_2/cp-0025.ckpt

Epoch 00030: saving model to trainin_2/cp-0030.ckpt

Epoch 00035: saving model to trainin_2/cp-0035.ckpt

Epoch 00040: saving model to trainin_2/cp-0040.ckpt

Epoch 00045: saving model to trainin_2/cp-0045.ckpt

Epoch 00050: saving model to trainin_2/cp-0050.ckpt


In [12]:
!ls {checkpoint_dir}

checkpoint                       cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001 cp-0030.ckpt.index
cp-0005.ckpt.index               cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001 cp-0035.ckpt.index
cp-0010.ckpt.index               cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001 cp-0040.ckpt.index
cp-0015.ckpt.index               cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001 cp-0045.ckpt.index
cp-0020.ckpt.index               cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001 cp-0050.ckpt.index
cp-0025.ckpt.index


In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [14]:
latest

'trainin_2/cp-0050.ckpt'

In [15]:
untrained_model = create_model()
untrained_model.load_weights(latest)

loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model, accuracy: {100*acc}")

1000/1000 - 0s - loss: 0.4818 - accuracy: 0.8780
Restored model, accuracy: 87.8000020980835


## Manually Save Weights

In [16]:
untrained_model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model, accuracy: {100*acc}")

1000/1000 - 0s - loss: 0.4818 - accuracy: 0.8780
Restored model, accuracy: 87.8000020980835


## Save Entire Model

In [17]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# !mkdir -p saved_model
model.save('saved_model/my_model')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 318us/sample - loss: 1.1860 - accuracy: 0.6670
Epoch 2/5
1000/1000 [==============================] - 0s 81us/sample - loss: 0.4224 - accuracy: 0.8920
Epoch 3/5
1000/1000 [==============================] - 0s 82us/sample - loss: 0.2947 - accuracy: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 81us/sample - loss: 0.2004 - accuracy: 0.9570
Epoch 5/5
1000/1000 [==============================] - 0s 79us/sample - loss: 0.1582 - accuracy: 0.9700
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [18]:
model.save('saved_model/my_model.h5')